# Advent of code 2023

Solutions are my own, if any external source including hints have been used it shall be mentioned and linked.


## Part1

For example:

    Time:      7  15   30
    Distance:  9  40  200

This document describes three races:

    The first race lasts 7 milliseconds. The record distance in this race is 9 millimeters.
    The second race lasts 15 milliseconds. The record distance in this race is 40 millimeters.
    The third race lasts 30 milliseconds. The record distance in this race is 200 millimeters.

    Don't hold the button at all (that is, hold it for 0 milliseconds) at the start of the race. The boat won't move; it will have traveled 0 millimeters by the end of the race.
    Hold the button for 1 millisecond at the start of the race. Then, the boat will travel at a speed of 1 millimeter per millisecond for 6 milliseconds, reaching a total distance traveled of 6 millimeters.
    Hold the button for 2 milliseconds, giving the boat a speed of 2 millimeters per millisecond. It will then get 5 milliseconds to move, reaching a total distance of 10 millimeters.
    Hold the button for 3 milliseconds. After its remaining 4 milliseconds of travel time, the boat will have gone 12 millimeters.
    Hold the button for 4 milliseconds. After its remaining 3 milliseconds of travel time, the boat will have gone 12 millimeters.
    Hold the button for 5 milliseconds, causing the boat to travel a total of 10 millimeters.
    Hold the button for 6 milliseconds, causing the boat to travel a total of 6 millimeters.
    Hold the button for 7 milliseconds. That's the entire duration of the race. You never let go of the button. The boat can't move until you let go of the button. Please make sure you let go of the button so the boat gets to move. 0 millimeters.


Since the current record for this race is 9 millimeters, there are actually 4 different ways you could win: you could hold the button for 2, 3, 4, or 5 milliseconds at the start of the race.

In the second race, you could hold the button for at least 4 milliseconds and at most 11 milliseconds and beat the record, a total of 8 different ways to win.

In the third race, you could hold the button for at least 11 milliseconds and no more than 19 milliseconds and still beat the record, a total of 9 ways you could win.

To see how much margin of error you have, determine the number of ways you can beat the record in each race; in this example, if you multiply these values together, you get 288 (4 * 8 * 9).

Determine the number of ways you could beat the record in each race. What do you get if you multiply these numbers together?

    


In [1]:
from __future__ import annotations
from dataclasses import dataclass
import math

@dataclass
class Race:
    time: int
    record: int

    def distance_traveled(self,hold:int)-> int:
        if hold == 0 or hold >= self.time:
            return 0
        time_traveled = self.time - hold
        return time_traveled * hold

        
    def number_records(self, batch):
        lo, hi = self.find_hold_limits(batch=batch)
        return sum( 
            self.distance_traveled(hold=hold) > self.record
            for hold in range(lo, hi+1)
        )
    
    def _find_lo(self, batch):
        left = 0  # Define the search range start point
        for i in range(left, self.time, batch):
            dist = self.distance_traveled(i)
            if dist > self.record:
                return max(0, i - batch) # clip if < 0 
            
    def _find_high(self, batch):
        right = self.time  # Define the search range start point
        for i in range(right, 0, -batch):
            dist = self.distance_traveled(i)
            if dist > self.record:
                return min(self.time, i + batch) # clip if over max time

    def find_hold_limits(self, batch):
        """reduces hold times search to a near point left and right
        the record area"""
        lo, hi = self._find_lo(batch=batch), self._find_high(batch=batch)
        return lo, hi
        
    @staticmethod
    def parse_race(puzzle:str, idx:int)->Race:
        """
        Time:      7  15   30
        Distance:  9  40  200
        """
        lines = puzzle.splitlines()
        for line in lines:
            name, values = line.split(":  ")
            values = values.split()
            
            if "Time" in name:
                time = int(values[idx])
            else:
                record = int(values[idx])
        return Race(time=time, record=record)
                
TEST = """Time:      7  15   30
Distance:  9  40  200"""

def parse_races(puzzle:str, part2=False)->list[Race]:
    """Parses all races in a puzzle input"""
    L = len(puzzle.splitlines()[0].split(": ")[-1].split())
    races = list()
    for idx in range(L):
        races.append(Race.parse_race(puzzle=puzzle, 
                                    idx=idx))
    if part2:
        time, record = "", ""
        for race in races:
            time += str(race.time)
            record += str(race.record)
        return Race(time=int(time), 
                    record=int(record)) 

    return races

def parts(races:list[Race], batch=2)->int:
    return math.prod(
        race.number_records(batch=batch)
        for race in races
        )

In [2]:
races = parse_races(puzzle=TEST)
assert parts(races=races) == 288
race = parse_races(puzzle=TEST, part2=True)
assert race.number_records(batch=10) == 71503

## Solutions

In [3]:
with open("puzzle_input/day06.txt") as file:
    puzzle = file.read()
races = parse_races(puzzle=puzzle)
race  = parse_races(puzzle=puzzle, part2=True)
print("part1", parts(races=races)) # 156156
print("part2", parts(races=[race], batch=10_000))

part1 293046


part2 35150181
